In [1]:
# Importaciones estándar
import os
import socket
import random
from itertools import islice

# Manejo de datos
import pandas as pd
import numpy as np

# Utilidades para limpieza de texto
import unicodedata
from unidecode import unidecode

# Visualización
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

# Procesamiento y estadísticas
from tqdm import tqdm
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler

#constantes
import constants
from constants import columna_valor_programa
from constants import columna_cupos_maximos
from constants import MINIMO_CUPOS_CERRADOS_SEDE

# NO EDITAR - NOTEBOOK SOLO PARA LECTURA

El propósito de este documento es servir como referencia para definir las clases

----

In [2]:
#Valor total del programa indexado para realizar la asignación de cupos
columna_valor_programa

'VALOR TOTAL DEL PROGRAMA INDEXADO'

-----

In [3]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

pd.set_option('display.float_format', '{:,.2f}'.format)

In [4]:
# Obtener el nombre del host
hostname = socket.gethostname()

hostname

'alejandro-lenovo'

In [5]:
# Obtener el nombre del host
hostname = socket.gethostname()

# Definir las rutas para cada computadora
path_pc1 = 'D:/OneDrive - Agencia Distrital para la Educación Superior, la Ciencia y la Tecnología - Atenea/Despacho/Investigaciones'
path_pc2 = 'C:/Users/JuliánNaranjo/OneDrive - Agencia Distrital para la Educación Superior, la Ciencia y la Tecnología - Atenea/2025 SAIGC/Índice ETDH'
path_pc3 = 'C:/Users/JULIA/OneDrive - Agencia Distrital para la Educación Superior, la Ciencia y la Tecnología - Atenea/Despacho/Investigaciones'

# Cambiar el directorio según el nombre del host
if hostname == 'DESKTOP-AEUL37C':
    os.chdir(path_pc1)
elif hostname == 'SGAINF-CND8085QVH-P26':
    os.chdir(path_pc2)
elif hostname == 'MSI':
    os.chdir(path_pc3)
else:
    print("Nombre de host no reconocido. Configura las rutas apropiadas para este dispositivo.")

# Imprime el directorio actual para verificar
print("Directorio actual:", os.getcwd())

Nombre de host no reconocido. Configura las rutas apropiadas para este dispositivo.
Directorio actual: /home/alejandro/Documentos/ATENEA/Despacho/Asignacion EFT


### Funciones

In [6]:
# Aplicar reemplazo seguro de los cno e ipo de los programas nuevos
def reemplazar_codigo(row):
    programa = row['nombre_programa']
    if programa in constants.programa_info:
        return pd.Series(constants.programa_info[programa])
    return pd.Series([row['cod_CNO'], row['IPO']])
    

In [7]:
def filtrar_ocupaciones_potenciales(data, ruta='antiguos'):
    '''
    Identifica las ocupaciones donde haya al menos un oferente con un valor no nulo
    en la columna 'ISOEFT_4d' (para ruta='antiguos') o en 'Puntaje (nuevos y cerrados)' (para ruta='nuevos').

    Un oferente corresponde a una fila del DataFrame. Una ocupación es considerada válida
    si al menos un oferente tiene un valor distinto de NaN en la columna correspondiente según el tipo de ruta.

    Parámetros:
    -----------
    data : pandas.DataFrame
        DataFrame que debe contener las columnas:
        - 'Ocupacion'
        - 'ISOEFT_4d' si ruta = 'antiguos'
        - 'Puntaje (nuevos y cerrados)' si ruta = 'nuevos'
    
    ruta : str, opcional (por defecto = 'antiguos')
        Indica si se debe usar el criterio de 'antiguos' o 'nuevos' para la validación.

    Retorna:
    --------
    pandas.DataFrame
        Subconjunto del DataFrame original que solo incluye las ocupaciones válidas.

    Errores:
    --------
    ValueError:
        Si el valor de 'ruta' no es 'antiguos' ni 'nuevos'.
    KeyError:
        Si faltan las columnas necesarias en el DataFrame.
    '''
    if ruta == 'antiguos':
        columna_valor = 'ISOEFT_4d'
    elif ruta == 'nuevos':
        columna_valor = 'Puntaje (nuevos y cerrados)'
    else:
        raise ValueError("El parámetro 'ruta' debe ser 'antiguos' o 'nuevos'.")

    # Validar que las columnas necesarias existen
    if columna_valor not in data.columns or 'Ocupacion' not in data.columns:
        raise KeyError(f"El DataFrame debe contener las columnas 'Ocupacion' y '{columna_valor}'.")

    # Filtrar ocupaciones válidas
    ocupaciones_validas = data.loc[data[columna_valor].notna(), 'Ocupacion'].unique()
    data_filtrada = data[data['Ocupacion'].isin(ocupaciones_validas)].copy()

    return data_filtrada


In [31]:
def calcular_ipo_ponderado(data, alfa=1, ponderar = True ):
    """
    Calcula el IPO ponderado a partir del número de cupos a ofertar y el índice IPO elevado a una potencia.

    Parámetros:
    -----------
    data : pandas.DataFrame
        DataFrame que debe contener las columnas 'numero_cupos_ofertar' y 'IPO'.
    alfa : float, opcional (por defecto=1.5)
        Exponente aplicado al valor de IPO para ponderar su influencia. 
        Debe ser un número mayor a 1.
    ponderar :  Boolean, opcional (por defecto = True)
        Distingue si se requiere ponderar por el numero de cupos a ofertar
    """
    
    # Validación del parámetro alfa
    if not isinstance(alfa, (int, float)):
        raise TypeError("El parámetro 'alfa' debe ser un número.")
    if alfa < 1:
        raise ValueError("El parámetro 'alfa' debe ser mayor o igual a 1.")
    
    # Cálculo del IPO ponderado
    Antiguos = data.copy()

    if ponderar:
        Antiguos['ipo_ponderado'] = Antiguos['numero_cupos_ofertar'] * (Antiguos['IPO'] ** alfa)
    else:
        alfa = 1
        beta = 0
        Antiguos['ipo_ponderado'] = (Antiguos['numero_cupos_ofertar']**beta) * (Antiguos['IPO'] ** alfa)
        
    
    return Antiguos


In [9]:
def calcular_recursos_por_cno(df, total_recursos, group = ['cod_CNO','Ocupacion']):
    """
    Calcula los recursos por cno segun la formula del paso 1 de la ruta de antiguos y nuevos. 
    
    Parámetros:
        df (pd.DataFrame): DataFrame con columnas 'cod_CNO', 'Ocupacion', 'ipo_ponderado', 'numero_cupos_ofertar', 'IPO'.
        total_recursos (float): Total de recursos disponibles para distribuir.

    Retorna:
        pd.DataFrame: DataFrame con las métricas agrupadas y columna 'recursosxcno' calculada.
    """
    # 1. Calcular la suma total de ipo_ponderado
    IO_total = df['ipo_ponderado'].sum()

    # 2. Agrupar por cod_CNO y Ocupacion
    IO_por_CNO = (
        df
        .groupby(group)
        .agg(
            ipo_ponderado=('ipo_ponderado', 'sum'),
            CUPOS=('numero_cupos_ofertar', 'sum'),
            IPO=('IPO', 'sum'),
            n_programas=('ipo_ponderado', 'count')
        )
        .reset_index()
    )

    # 3. Calcular participación
    IO_por_CNO['IO_part'] = IO_por_CNO['ipo_ponderado'] / IO_total

    # 4. Asignar recursos
    IO_por_CNO['recursosxcno'] = IO_por_CNO['IO_part'] * total_recursos

    IO_por_CNO = IO_por_CNO[ group + ['recursosxcno','n_programas']]

    return IO_por_CNO


In [10]:
def ordenar_ocupaciones_por_isoeft(df):
    """
    Ordena un DataFrame por ['cod_CNO', 'Ocupacion', 'IPO', 'ISOEFT_4d'],
    asegurando que las filas con NaN en 'ISOEFT_4d' queden al final del DataFrame completo.

    Parámetros:
        df (pd.DataFrame): El DataFrame a ordenar.

    Retorna:
        pd.DataFrame: DataFrame ordenado con NaNs al final.
    """
    # Separar por presencia de NaN en ISOEFT_4d
    sin_nan = df[df['ISOEFT_4d'].notna()]
    con_nan = df[df['ISOEFT_4d'].isna()]

    #Columnas para ordenar
    columnas = [
        'IPO',
        'cod_CNO',
        'Ocupacion',
        'ISOEFT_4d',
        columna_valor_programa,
        "numero_cupos_ofertar",
        "duracion_horas_programa"
    ]

    orden = [
        False,
        True,
        True,
        False,
        True,
        False,
        True
    ]
    
    # Ordenar las filas válidas
    sin_nan = sin_nan.sort_values(
        columnas, 
        ascending= orden
    )

    # Concatenar y devolver
    return pd.concat([sin_nan, con_nan], ignore_index=True)


In [11]:
def ordenar_sedes_programas(df, usar_cod_cno=True):
    """
    Ordena los programas dentro del DataFrame según criterios establecidos.
    Si usar_cod_cno es True, cod_CNO se usará como primer criterio de orden.

    Criterios de orden:
    1. (Opcional) cod_CNO
    2. Mayor puntaje
    3. Mayor número de cupos ofertados
    4. Mayor meta de vinculación
    5. Menor costo
    6. Menor duración

    Parámetros:
        df (pd.DataFrame): DataFrame con columnas relevantes.
        usar_cod_cno (bool): Si True, incluye cod_CNO como primer criterio.

    Retorna:
        pd.DataFrame: DataFrame ordenado según los criterios establecidos.
    """
    columnas = []
    orden = []

    if usar_cod_cno:
        columnas.append('cod_CNO')
        orden.append(True)

    columnas += [
        'Puntaje (nuevos y cerrados)',
        'numero_cupos_ofertar',
        'Meta de vinculación',
        columna_valor_programa,
        'duracion_horas_programa'
    ]

    orden += [False, False, False, True, True]

    df_ordenado = df.sort_values(by=columnas, ascending=orden).reset_index(drop=True)
    return df_ordenado



In [12]:
def asignar_recursos_y_cupos_viejos(data):
    """
    Asigna cupos y recursos por ocupación según los lineamientos de la Ruta Antigua, paso 2

    Retorna un resumen por ocupación con cupos y recursos asignados, y los saldos no utilizados.
    """
   
    data = data.copy()
    # Paso 1: Crear nueva columna para asignación
    data['cupos_asignados_2E'] = 0

    # Paso 2: Iterar por grupo de ocupación para asignar los recursos disponibles
    for (cod_cno, Ocupacion), grupo in data.groupby(['cod_CNO', 'Ocupacion']):
        recurso_por_dispersar = grupo['recursosxcno'].iloc[0]
        saldo = recurso_por_dispersar
        indices = grupo.index

        for i in indices:
            costo_unitario = data.loc[i, columna_valor_programa]
            cupos_disp = data.loc[i, 'numero_cupos_ofertar']

            if pd.isna(costo_unitario) or costo_unitario == 0:
                continue

            recurso_necesario = cupos_disp * costo_unitario

            if saldo >= recurso_necesario:
                data.loc[i, 'cupos_asignados_2E'] = cupos_disp
                saldo -= recurso_necesario
            else:
                # Ver si se puede financiar al menos un cupo
                cupos_asignables = saldo // costo_unitario
                data.loc[i, 'cupos_asignados_2E'] = cupos_asignables
                saldo -= cupos_asignables * costo_unitario
                break

    # Paso 3: Calcular recursos efectivamente asignados por programa
    data['recurso_asignado_2E'] = data['cupos_asignados_2E'] * data[columna_valor_programa]

    # Paso 4: Agrupar para obtener resumen de asignaciones por ocupacion
    asignacion_por_ocupacion_ant = data.groupby(['cod_CNO', 'Ocupacion']).agg(
        recurso_asignado_2E=('recurso_asignado_2E', 'sum'),
        cupos_asignados_2E=('cupos_asignados_2E', 'sum')
    ).reset_index()

    # Paso 5: Obtener recursos originales y número de cupos ofertados por las instituciones
    recursos_por_ocupacion = data.groupby(['cod_CNO', 'Ocupacion']).agg(
        recursosxcno=('recursosxcno', 'first'),
        numero_cupos_ofertar=('numero_cupos_ofertar', 'sum')
    ).reset_index()

    # Paso 6: Unir ambas tablas
    asignacion_por_ocupacion_ant = asignacion_por_ocupacion_ant.merge(
        recursos_por_ocupacion, on=['cod_CNO', 'Ocupacion']
    )

    # Paso 7: Calcular saldos no asignados
    asignacion_por_ocupacion_ant['Saldo_No_Asignado_2E'] = (
        asignacion_por_ocupacion_ant['recursosxcno'] - asignacion_por_ocupacion_ant['recurso_asignado_2E']
    )

    asignacion_por_ocupacion_ant['cupos_no_asignados_2E'] = (
        asignacion_por_ocupacion_ant['numero_cupos_ofertar'] - asignacion_por_ocupacion_ant['cupos_asignados_2E']
    )

    return data, asignacion_por_ocupacion_ant


In [13]:
def asignar_tercera_etapa(Antiguos, asignacion_por_ocupacion_ant):
    """
    Asigna recursos sobrantes de la segunda etapa a programas priorizados en una tercera etapa,
    usando una bolsa común. Actualiza el DataFrame original con asignaciones adicionales.
    """
    saldo_comun_3E = asignacion_por_ocupacion_ant['Saldo_No_Asignado_2E'].sum()
    
    Antiguos = Antiguos.copy()
    
    Antiguos['cupos_asignados_3E'] = 0
    Antiguos['recurso_asignado_3E'] = 0

    Antiguos = ordenar_ocupaciones_por_isoeft(Antiguos)

    for idx, row in Antiguos.iterrows():
        costo = row[columna_valor_programa]
        cupos = row['numero_cupos_ofertar']
        if pd.isna(costo) or costo <= 0 or pd.isna(cupos) or cupos <= 0:
            continue

        recurso_necesario = costo * cupos
        
        if saldo_comun_3E >= recurso_necesario:
            Antiguos.at[idx, 'cupos_asignados_3E'] = cupos
            Antiguos.at[idx, 'recurso_asignado_3E'] = recurso_necesario
            saldo_comun_3E -= recurso_necesario
        else:
            cupos_posibles = saldo_comun_3E // costo
            if cupos_posibles >= 1:
                recurso_asignado = cupos_posibles * costo
                Antiguos.at[idx, 'cupos_asignados_3E'] = cupos_posibles
                Antiguos.at[idx, 'recurso_asignado_3E'] = recurso_asignado
                saldo_comun_3E -= recurso_asignado
            else:
                break

    Antiguos['Total_Cupos_Asignados'] = Antiguos['cupos_asignados_2E'] + Antiguos['cupos_asignados_3E']
    Antiguos['Total_Recurso_Asignado'] = Antiguos['recurso_asignado_2E'] + Antiguos['recurso_asignado_3E']
    Antiguos['Saldo_Remanente_3E'] = saldo_comun_3E
    return Antiguos

In [14]:
def asignar_recursos_y_cupos_nuevos(data):
    
    data = data.copy()
    
    # Inicializar columna para cupos asignados
    data['cupos_asignados_2E'] = 0
    
    # Asignación iterativa por cod_CNO
    for cod_cno, grupo in data.groupby('cod_CNO'):
        
        recurso_total = grupo['recursosxcno'].iloc[0] #mismo recurso para cada ocupacion
        saldo = recurso_total
        indices = grupo.index
    
        for i in indices:
            costo_unitario = data.loc[i, columna_valor_programa]
            cupos_disp = data.loc[i, 'numero_cupos_ofertar']
            if pd.isna(costo_unitario) or costo_unitario == 0:
                continue
    
            recurso_necesario = cupos_disp * costo_unitario
    
            if saldo >= recurso_necesario:
                data.loc[i, 'cupos_asignados_2E'] = cupos_disp
                saldo -= recurso_necesario
            else:
                cupos_asignables = saldo // costo_unitario
                data.loc[i, 'cupos_asignados_2E'] = cupos_asignables
                saldo -= cupos_asignables * costo_unitario
                break
    
    # Paso 5: Calcular recursos asignados
    data['recurso_asignado_2E'] = data['cupos_asignados_2E'] * data[columna_valor_programa]
    
    # Paso 6: Agrupar para obtener resumen de asignaciones por ocupacion
    asignacion_por_ocupacion_nuevos = data.groupby('cod_CNO').agg(
        recurso_asignado_2E=('recurso_asignado_2E', 'sum'),
        cupos_asignados_2E=('cupos_asignados_2E', 'sum')
    ).reset_index()
    
    
    # Paso 5: Obtener recursos originales y número de cupos ofertados por las instituciones
    recursos_por_ocupacion = data.groupby('cod_CNO').agg(
        recursosxcno=('recursosxcno', 'first'),
        numero_cupos_ofertar=('numero_cupos_ofertar', 'sum')
    ).reset_index()
    
    
    # Paso 7: Agregar recursos estimados y calcular saldo no asignado
    asignacion_por_ocupacion_nuevos = asignacion_por_ocupacion_nuevos.merge(
        recursos_por_ocupacion, on='cod_CNO'
    )
    
    asignacion_por_ocupacion_nuevos['Saldo_No_Asignado_2E'] = (
        asignacion_por_ocupacion_nuevos['recursosxcno'] - asignacion_por_ocupacion_nuevos['recurso_asignado_2E']
    )
    
    asignacion_por_ocupacion_nuevos['cupos_no_asignados_2E'] = (
        asignacion_por_ocupacion_nuevos['numero_cupos_ofertar'] - asignacion_por_ocupacion_nuevos['cupos_asignados_2E']
    )

    return data, asignacion_por_ocupacion_nuevos

In [15]:
def asignar_recursos_grupos_cerrados(Grupos_Cerrados, total_recursos_cerrados):
    """
    Asigna recursos a programas de grupos cerrados hasta agotar el saldo total disponible.

    Parámetros:
        Grupos_Cerrados (pd.DataFrame): DataFrame con la información de los programas.
        total_recursos_cerrados (float): Monto total disponible para asignar.

    Retorna:
        pd.DataFrame: DataFrame con columnas de cupos y recursos asignados, y el saldo restante.
    """
    Grupos_Cerrados = Grupos_Cerrados.copy()
    
    Grupos_Cerrados['cupos_asignados_2E'] = 0  
    Grupos_Cerrados['recurso_asignado_2E'] = 0.0
    Grupos_Cerrados['saldo_total_remanente'] = 0.0


    saldo_total = total_recursos_cerrados

    for idx, row in Grupos_Cerrados.iterrows():
        costo = row[columna_valor_programa]
        cupos = min(row['numero_cupos_ofertar'], row[columna_cupos_maximos])

        if pd.isna(costo) or costo <= 0 or pd.isna(cupos) or cupos <= 0:
            Grupos_Cerrados.at[idx, 'saldo_total_remanente'] = saldo_total
            continue

        recurso_necesario = costo * cupos

        if saldo_total >= recurso_necesario:
            Grupos_Cerrados.at[idx, 'cupos_asignados_2E'] = cupos
            Grupos_Cerrados.at[idx, 'recurso_asignado_2E'] = recurso_necesario
            saldo_total -= recurso_necesario
        else:
            
            cupos_posibles = saldo_total // costo
            recurso_asignado = cupos_posibles * costo
            
            Grupos_Cerrados.at[idx, 'cupos_asignados_2E'] = cupos_posibles
            Grupos_Cerrados.at[idx, 'recurso_asignado_2E'] = recurso_asignado
            saldo_total -= recurso_asignado

        Grupos_Cerrados.at[idx, 'saldo_total_remanente'] = saldo_total

        if saldo_total <= 0:
            break

    return Grupos_Cerrados, saldo_total

In [16]:
def asignar_recursos_hasta_validacion(Grupos_Cerrados, recursos_por_dispersar, MINIMO_CUPOS_CERRADOS_SEDE=50):
    """
    Ejecuta la asignación de recursos, eliminando iterativamente instituciones que no alcanzan el mínimo de cupos.

    Retorna:
        Grupos_Cerrados_final (pd.DataFrame)
        instituciones_eliminadas (list[str])
    """
    instituciones_eliminadas = [] #Aqui se guardaran las instituciones que no cumplieron con el minimo de 50 cupos
    filas_eliminadas = []  # Aquí se guardarán las filas eliminadas para concatenar al final
    continuar = True

    saldo_remanente = recursos_por_dispersar

    while continuar:
        # Asignar recursos
        Grupos_Cerrados, saldo_remanente = asignar_recursos_grupos_cerrados(Grupos_Cerrados, saldo_remanente)

        # Identificar instituciones a eliminar
        instituciones_a_eliminar = []

        for institucion in Grupos_Cerrados['nombre_institucion'].unique():
            df_inst = Grupos_Cerrados[Grupos_Cerrados['nombre_institucion'] == institucion]
            
            #evaluar la suma de los cupos asignados a la institucion
            total_cupos = df_inst['cupos_asignados_2E'].sum()
            #evaluar la suma dispersada a la institucion
            recursos_disperados_inst = df_inst['recurso_asignado_2E'].sum()
            
            if 0 < total_cupos < MINIMO_CUPOS_CERRADOS_SEDE:
                
                instituciones_a_eliminar.append(institucion)
                #reintregamos al saldolos recursos que se le habia dado a la institucion
                saldo_remanente += recursos_disperados_inst

        if instituciones_a_eliminar:
            
            # Seleccionar filas a eliminar
            filas_a_eliminar = Grupos_Cerrados[Grupos_Cerrados['nombre_institucion'].isin(instituciones_a_eliminar)].copy()
            
            # Actualizar sus valores
            filas_a_eliminar['cupos_asignados_2E'] = 0
            filas_a_eliminar['recurso_asignado_2E'] = 0.0
            filas_a_eliminar['saldo_total_remanente'] = saldo_remanente
            
            # Guardar filas actualizadas
            filas_eliminadas.append(filas_a_eliminar)

            # Añadir las instituciones a eliminar a la lista
            instituciones_eliminadas.extend(instituciones_a_eliminar)
            
            # Eliminar las instituciones del DataFrame principal
            Grupos_Cerrados = Grupos_Cerrados[
                ~Grupos_Cerrados['nombre_institucion'].isin(instituciones_a_eliminar)
            ].reset_index(drop=True)
        else:
            continuar = False

    # Reagregar las filas eliminadas al final del DataFrame
    if filas_eliminadas:
        Grupos_Cerrados = pd.concat([Grupos_Cerrados] + filas_eliminadas, ignore_index=True)

    return Grupos_Cerrados, instituciones_eliminadas


In [17]:
def reasignar_remanente(df_remanente, bolsa, columna_cupos= 'cupos_asignados_2E'):
    """
    Reasigna recursos disponibles (remanente) a los cupos restantes de un DataFrame, 
    hasta agotar el saldo o completar los cupos pendientes.
    """
    df_remanente = df_remanente.copy()
    
    #df_remanente['remanente_asignado'] = 0
    
    saldo_total = bolsa

    for idx, row in df_remanente.iterrows():
        cupos_restantes =  row['numero_cupos_ofertar'] - row[columna_cupos] 
        costo = row[columna_valor_programa]

        if pd.isna(costo) or costo <= 0 or pd.isna(cupos_restantes) or cupos_restantes <= 0:
            df_remanente.at[idx, 'saldo_total_remanente'] = saldo_total
            continue

        recurso_necesario = costo * cupos_restantes

        if saldo_total >= recurso_necesario:
            df_remanente.at[idx, 'cupos_asignados_remanente'] = cupos_restantes
            df_remanente.at[idx, 'recurso_asignado_remanente'] = recurso_necesario
            saldo_total -= recurso_necesario
        else:
            cupos_posibles = saldo_total // costo
            recurso_asignado = cupos_posibles * costo

            df_remanente.at[idx, 'cupos_asignados_remanente'] = cupos_posibles
            df_remanente.at[idx, 'recurso_asignado_remanente'] = recurso_asignado
            saldo_total -= recurso_asignado

        df_remanente.at[idx, 'saldo_total_remanente'] = saldo_total

        if saldo_total <= 0:
            break

    return df_remanente, saldo_total

### Lectura de datos

In [18]:
#Cargar los datos
Programas_EFT_Oferta = pd.read_pickle("Asignacion de cupos/Base final - Oferta Activa.pkl")
Programas_EFT = pd.read_excel("Asignacion de cupos/Habilitados final 26052025.xlsx")

In [19]:
#Limpieza de los datos

# Seleccionar las columnas de interes
Programas_EFT_Oferta = Programas_EFT_Oferta[constants.columnas_programas_eft_oferta]

#Renombrar algunas columnas
Programas_EFT = Programas_EFT.rename(
    columns= constants.nombre_columnas_mapping
)

#Cambiar el tipo de los datos
Programas_EFT = Programas_EFT.astype(constants.tipo_columnas_mapping)
Programas_EFT_Oferta = Programas_EFT_Oferta.astype(constants.tipo_columnas_mapping)

In [20]:
# Unir Programas_EFT y Programas_EFT_Oferta
Programas_EFT = Programas_EFT.merge(
    Programas_EFT_Oferta,
    how='left',
    on=['CODIGO_PROGRAMA']
)

----

### Descripcion de los datos

In [21]:
#CNO: Clasificacion nancional de ocupaciones
# Suponiendo que 'cod_CNO' proviene de Programas_EFT_Oferta y no está en Programas_EFT original
cruzaron = Programas_EFT['cod_CNO'].notna().sum()
no_cruzaron = Programas_EFT['cod_CNO'].isna().sum()

print(f"Registros que cruzaron: {cruzaron}")
print(f"Registros que NO cruzaron: {no_cruzaron}")

Registros que cruzaron: 39
Registros que NO cruzaron: 6


In [22]:
sin_cno_antiguos = Programas_EFT[
    Programas_EFT['cod_CNO'].isna() &
    (Programas_EFT['Ruta habilitada'] == "Antiguos")
]

print(f"Numero de programas antiguos sin cno: {sin_cno_antiguos.shape[0]}")

Numero de programas antiguos sin cno: 0


In [23]:
Programas_EFT['Ruta habilitada'].value_counts()

Ruta habilitada
Antiguos           25
Nuevos             10
Grupos cerrados    10
Name: count, dtype: int64

In [41]:
ruta_labels = Programas_EFT['Ruta habilitada'].unique()

# Crear un diccionario con un DataFrame por cada valor de ruta
rutas_dict = {
    ruta: Programas_EFT.loc[Programas_EFT['Ruta habilitada'] == ruta].copy()
    for ruta in ruta_labels
}

# Ejemplos de acceso:
Antiguos = rutas_dict.get("Antiguos")
Nuevos = rutas_dict.get("Nuevos")
No_Habilitado = rutas_dict.get("NO HABILITADO")
Grupos_Cerrados = rutas_dict.get("Grupos cerrados")

-----

# Ruta: Antiguos

Pasos:
1. PASO 1: Asignación por ocupación proporcional al IPO Ponderado
2. PASO 2: Asignación de recursos en función del resultado de ISOEFT
3. PASO 3: Bolsa común con recursos excedentes para completar recursos.

Variación para la sección del IPO:

**Bolsas por ocupacion** sin tener en cuenta la oferta de cupos, esto es que al menos **exista un oferente potencial** en **cada ocupación** para que se genere una bolsa (un oferente potencial implica que esté bien sea en el grupo de antiguos o nuevos, y **tenga segun sea el caso ISOEFT** o **el puntaje**)

In [42]:
#Validación de que no existan ocupaciones con IPO menor al minimo (ie., 0.47)
min_value = Antiguos['IPO'].min()
print("Min:", min_value)
assert min_value >= 0.47, f"Min value {min_value} is less than 0.47"

Min: 0.4773236184370968


### Paso 1: Asignación por ocupación proporcional al IPO Ponderado

In [43]:
assert Antiguos['IPO'].notna().all(), "Existen NaNs in IPO"

In [44]:
assert Antiguos['numero_cupos_ofertar'].notna().all(), "Existen NaNs in 'numero_cupos_ofertar'"

In [45]:
# Calculo del IPO ponderado
Antiguos = calcular_ipo_ponderado(Antiguos, alfa = 1)

In [46]:
total_recursos_antiguos = constants.recursos_por_ruta['antiguos']
print(f"Recursos totales para disperar en la ruta de Antiguos: {total_recursos_antiguos:,.0f}")

Recursos totales para disperar en la ruta de Antiguos: 1,320,000,000


In [47]:
IO_por_CNO = calcular_recursos_por_cno(Antiguos, total_recursos_antiguos)
IO_por_CNO.head()

,cod_CNO,Ocupacion,recursosxcno,n_programas
0,1231,Asistentes contables,"107,569,094.89",2
1,1345,Auxiliares administrativos en salud,"111,126,632.02",2
2,2242,Técnicos en electrónica,"45,308,756.34",1
3,2281,Técnicos en tecnologías de la información,"61,535,606.34",1
4,3311,Auxiliares en enfermería,"53,180,231.95",1


In [48]:
IO_por_CNO.to_excel("Export/recursos_por_CNO.xlsx")

In [49]:
suma_recursos_por_cno = IO_por_CNO['recursosxcno'].sum()
print(f"Total de Recursos por CNO: {suma_recursos_por_cno:,.0f}")

# verificar que los recursos asignados por ocupaciones sean iguales a los recursos disponbiles
assert suma_recursos_por_cno == total_recursos_antiguos , f"Los recursos dispersados no son iguales al total de recursos disponibles"

Total de Recursos por CNO: 1,320,000,000


In [50]:
#Agregarle la columna recrusosxcno al df Antiguos
# Nota: Puede haber mas de un programa en una ocupacion, recursosxcno tiene valores duplicados
Antiguos = Antiguos.merge(
    # Le agreg
    IO_por_CNO[['cod_CNO', 'Ocupacion','recursosxcno']],
    on = ['cod_CNO','Ocupacion'],
    how='left'
)

### PASO 2: Asignación de recursos en función del resultado de ISOEFT

a) Ordenamiento de programas: Los programas dentro de cada ocupación se organizan en orden descendente, según su puntaje de desempeño en el ISOEFT, priorizando aquellos con mejor desempeño.

b) Asignación secuencial de recursos: La distribución de los recursos se realiza de manera ordenada, asignándolos a los programas ordenados y agrupados Este proceso sigue una secuencia estructurada en la que los cupos se asignan hasta que ocurra alguna de las siguientes condiciones:

- Se agoten los cupos ofertados dentro de la ocupación
- Se agoten los recursos asignados para la ocupación
- El saldo disponible no sea suficiente para financiar un cupo completo en ningún programa de la ocupación (siguiendo el ordenamiento por ISOEFT del programa)

c) Si, al finalizar el proceso, quedan recursos sin asignar, estos se trasladarán a una bolsa común, que será distribuida en la siguiente fase del proceso de asignación de recursos (Paso 3)

**a)** Ejecutar paso 2.a

El manual no es claro cual ocupación va primero:

"Ordenamiento de programas: Los programas dentro de cada ocupación se organizan en orden descendente, según su puntaje de desempeño en el ISOEFT, priorizando aquellos con mejor desempeño".

In [51]:
assert Antiguos['ISOEFT_4d'].notna().all(), "Existen NaNs in ISOEFT_4d"

AssertionError: Existen NaNs in ISOEFT_4d

In [52]:
assert Antiguos[columna_valor_programa].notna().all(), "Existen NaNs in valor programa indexado"

In [53]:
Antiguos = ordenar_ocupaciones_por_isoeft(Antiguos)

In [54]:
columnas_mostrar = [
    'cod_CNO',
    'Ocupacion',
    'IPO',
    'ISOEFT_4d',
    'CODIGO_INSTITUCION',
    'CODIGO_PROGRAMA',
    'nombre_programa',
    'recursosxcno', 
    'numero_cupos_ofertar'
]
Antiguos[columnas_mostrar].head()

,cod_CNO,Ocupacion,IPO,ISOEFT_4d,CODIGO_INSTITUCION,CODIGO_PROGRAMA,nombre_programa,recursosxcno,numero_cupos_ofertar
0,2281,Técnicos en tecnologías de la información,0.67,0.39,9025,42561,TÉCNICO LABORAL EN PROCESAMIENTO Y DIGITACIÓN ...,"61,535,606.34",83
1,1345,Auxiliares administrativos en salud,0.62,0.61,975,43512,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO EN ...,"111,126,632.02",80
2,1345,Auxiliares administrativos en salud,0.62,0.53,1236,41020,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO EN ...,"111,126,632.02",80
3,3315,Auxiliares en servicios farmacéuticos,0.60,0.54,1236,41022,TÉCNICO LABORAL EN AUXILIAR EN SERVICIOS FARMA...,"106,929,561.91",80
4,3315,Auxiliares en servicios farmacéuticos,0.60,0.53,975,43511,TÉCNICO LABORAL EN AUXILIAR EN SERVICIOS FARMA...,"106,929,561.91",80


**b)** Ejecutar Paso 2.b

In [55]:
Antiguos, asignacion_por_ocupacion_ant = asignar_recursos_y_cupos_viejos(Antiguos)

In [56]:
asignacion_por_ocupacion_ant.head()

,cod_CNO,Ocupacion,recurso_asignado_2E,cupos_asignados_2E,recursosxcno,numero_cupos_ofertar,Saldo_No_Asignado_2E,cupos_no_asignados_2E
0,1231,Asistentes contables,"106,624,039.03",29,"107,569,094.89",200,"945,055.86",171
1,1345,Auxiliares administrativos en salud,"110,572,617.97",17,"111,126,632.02",160,"554,014.05",143
2,2242,Técnicos en electrónica,"42,623,849.96",10,"45,308,756.34",75,"2,684,906.38",65
3,2281,Técnicos en tecnologías de la información,"57,725,449.20",12,"61,535,606.34",83,"3,810,157.14",71
4,3311,Auxiliares en enfermería,"49,340,892.24",5,"53,180,231.95",100,"3,839,339.71",95


### PASO 3: Bolsa común con recursos excedentes para completar recursos

Los recursos excedentes serán agrupados en una bolsa común y asignados siguiendo lo estipulado en el paso 2. Primero, se ordenarán las ocupaciones según el IPO, y dentro de cada ocupación, los programas se priorizarán por su ISOEFT. La asignación de recursos continuará siguiendo este orden hasta que se financien todos los cupos de un programa, se agoten los recursos de la bolsa común o el saldo disponible no permita financiar al menos un cupo completo.

Criterios de priorización:
En caso de que haya programas que tengan el mismo valor del ISOEFT, los criterios de priorización para la asignación de recursos se realizarán en el siguiente orden:

- i. Se dará preferencia a los programas con el menor valor de matrícula
- ii. En el caso de que tengan el mismo valor de matrícula se priorizará el programa para el cual se hayan ofertado más cupos.
- iii. En caso de los programas tengan el mismo valor de matrícula y el mismo número de cupos ofertados, se priorizará el programa con el menor número de horas.

In [57]:
assert Antiguos[columna_valor_programa].notna().all(), "Existen NaNs in valor programa indexado"

In [58]:
Antiguos = asignar_tercera_etapa(Antiguos, asignacion_por_ocupacion_ant)

/tmp/ipykernel_28100/3855908659.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28862724.599999998' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Antiguos.at[idx, 'recurso_asignado_3E'] = recurso_asignado


In [59]:
columnas_mostrar = [
    'cod_CNO',
    'Ocupacion',
    'IPO',
    'ISOEFT_4d',
    'CODIGO_INSTITUCION',
    'CODIGO_PROGRAMA',
    'recursosxcno', 
    'numero_cupos_ofertar',
    'cupos_asignados_2E',
    'recurso_asignado_2E',
    'cupos_asignados_3E',
    'recurso_asignado_3E',
    'Total_Cupos_Asignados',
    "Saldo_Remanente_3E"
]
Antiguos[columnas_mostrar].head()

,cod_CNO,Ocupacion,IPO,ISOEFT_4d,CODIGO_INSTITUCION,CODIGO_PROGRAMA,recursosxcno,numero_cupos_ofertar,cupos_asignados_2E,recurso_asignado_2E,cupos_asignados_3E,recurso_asignado_3E,Total_Cupos_Asignados,Saldo_Remanente_3E
0,2281,Técnicos en tecnologías de la información,0.67,0.39,9025,42561,"61,535,606.34",83,12,"57,725,449.20",6,"28,862,724.60",18,"4,390,248.13"
1,1345,Auxiliares administrativos en salud,0.62,0.61,975,43512,"111,126,632.02",80,17,"110,572,617.97",0,0.00,17,"4,390,248.13"
2,1345,Auxiliares administrativos en salud,0.62,0.53,1236,41020,"111,126,632.02",80,0,0.00,0,0.00,0,"4,390,248.13"
3,3315,Auxiliares en servicios farmacéuticos,0.60,0.54,1236,41022,"106,929,561.91",80,16,"106,447,062.42",0,0.00,16,"4,390,248.13"
4,3315,Auxiliares en servicios farmacéuticos,0.60,0.53,975,43511,"106,929,561.91",80,0,0.00,0,0.00,0,"4,390,248.13"


In [60]:
Antiguos.to_excel("Export/Antiguos.xlsx", index=False)
asignacion_por_ocupacion_ant.to_excel("Export/AntiguosxOcupación.xlsx", index=False)

------

# Ruta: Nuevos

Pasos:
1. PASO 1: Asignación por ocupación proporcional al IPO Ponderado
2. PASO 2: Asignación de puntaje a partir de los elementos puntuables
3. PASO 3: Asignación de recursos para la selección de cupos

In [61]:
#Limpieza datos

#reemplazar los codigos cno e ipo de manera segura
Nuevos[['cod_CNO', 'IPO']] = Nuevos.apply(reemplazar_codigo, axis=1)
# Forzar tipo entero en cod_CNO (permitiendo nulos)
Nuevos['cod_CNO'] = pd.to_numeric(Nuevos['cod_CNO'], errors='coerce').astype('Int64')

In [62]:
total_recursos_nuevos = constants.recursos_por_ruta['nuevos']
print(f"Recursos totales para disperar en la ruta de Nuevos: {total_recursos_nuevos:,.0f}")

Recursos totales para disperar en la ruta de Nuevos: 990,000,000


### Paso 1 

In [63]:
#Nuevos['ipo_ponderado'] = Nuevos['numero_cupos_ofertar'] * Nuevos['IPO'] 
Nuevos = calcular_ipo_ponderado(Nuevos, alfa = 1)

In [64]:
IO_por_CNO_nuevo = calcular_recursos_por_cno(Nuevos, total_recursos_nuevos, group = ['cod_CNO'])
IO_por_CNO_nuevo

,cod_CNO,recursosxcno,n_programas
0,2281,"589,967,275.83",4
1,2321,"69,818,825.16",1
2,3311,"60,442,068.24",1
3,6233,"66,270,863.08",1
4,6374,"70,705,815.68",1
5,6642,"69,185,260.50",1
6,8325,"63,609,891.52",1


In [65]:
IO_por_CNO_nuevo.to_excel("Export/recursos_por_CNO_nuevos.xlsx")

In [66]:
suma_recursos_por_cno = IO_por_CNO_nuevo['recursosxcno'].sum()
print(f"Total de Recursos para dispersar: {suma_recursos_por_cno:,.0f}")

# verificar que los recursos asignados por ocupaciones sean iguales a los recursos disponbiles
assert suma_recursos_por_cno == total_recursos_nuevos , f"Los recursos dispersados no son iguales al total de recursos disponibles"

Total de Recursos para dispersar: 990,000,000


In [67]:
Nuevos = Nuevos.merge(
    IO_por_CNO_nuevo[['cod_CNO','recursosxcno']],
    on=['cod_CNO'],
    how='left'
)

### Paso 2
La data viene con la columna de puntajes

### Paso 3

In [68]:
# Paso 2: Ordenar programas por prioridad (dentro de cada cod_CNO)
Nuevos =  ordenar_sedes_programas(Nuevos)

In [69]:
Nuevos, asignacion_por_ocupacion_nuevos = asignar_recursos_y_cupos_nuevos(Nuevos)

In [70]:
Nuevos.to_excel("Export/Nuevos.xlsx", index=False)
asignacion_por_ocupacion_nuevos.to_excel("Export/NuevosxOcupación.xlsx", index=False)

In [71]:
asignado_nuevos = asignacion_por_ocupacion_nuevos['recurso_asignado_2E'].sum()
print(f"Recursos asignados en Nuevos: {asignado_nuevos:,.0f}")

Recursos asignados en Nuevos: 972,601,089


-----

### Grupos Cerrados

### Paso 1 

La data viene con los puntajes

### Paso 2: Asignación de recursos para la selección de cupos

In [72]:
# Ordenar programas por prioridad
Grupos_Cerrados = ordenar_sedes_programas(Grupos_Cerrados, usar_cod_cno = False)

In [73]:
total_recursos_cerrados = constants.recursos_por_ruta['cerrados']
print(f"Recursos totales para disperar en la ruta de Cerrados: {total_recursos_cerrados:,.0f}")

Recursos totales para disperar en la ruta de Cerrados: 990,000,000


In [74]:
# Se calcula el monto a dispersar sin considera el filtro de 50 programas minimo
Grupos_Cerrados_final, saldo_cerrados =  asignar_recursos_grupos_cerrados(
    Grupos_Cerrados,
    total_recursos_cerrados
)

In [75]:
Grupos_Cerrados_final[['nombre_institucion',columna_valor_programa,'Puntaje (nuevos y cerrados)',columna_cupos_maximos,'cupos_asignados_2E', 'recurso_asignado_2E', 'saldo_total_remanente']]

,nombre_institucion,VALOR TOTAL DEL PROGRAMA INDEXADO,Puntaje (nuevos y cerrados),Número máximo de cupos por grupos,cupos_asignados_2E,recurso_asignado_2E,saldo_total_remanente
0,KUEPA EDUTECH,"4,810,454.10",88.75,888,110,"529,149,951.00","460,850,049.00"
1,KUEPA EDUTECH,"4,810,454.10",88.75,277,40,"192,418,164.00","268,431,885.00"
2,KUEPA EDUTECH,"4,810,454.10",88.75,69,20,"96,209,082.00","172,222,803.00"
3,CESDE BOGOTÁ,"6,232,522.68",56.75,90,27,"168,278,112.33","3,944,690.67"
4,CESDE BOGOTÁ,"5,406,154.01",56.75,60,0,0.00,"3,944,690.67"
5,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,194,543.61",56.25,420,0,0.00,"3,944,690.67"
6,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,504,271.65",56.25,420,0,0.00,"3,944,690.67"
7,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",56.25,612,0,0.00,"3,944,690.67"
8,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",56.25,612,0,0.00,"3,944,690.67"
9,MEDISED INSTITUCION DE EDUCACION PARA EL TRABA...,"6,839,631.92",32.38,100,0,0.00,"3,944,690.67"


In [76]:
asignado_cerrados =  Grupos_Cerrados_final['recurso_asignado_2E'].sum()
print(f"Recursos asignados en Cerrados: {asignado_cerrados:,.0f}")

Recursos asignados en Cerrados: 986,055,309


In [77]:
Grupos_Cerrados_final.to_excel("Export/GCerrados.xlsx", index=False)

----

In [78]:
recursos_remanente_antiguos = Antiguos['Saldo_Remanente_3E'].loc[0] 
print(f"Bolsa remanente Antiguos: {recursos_remanente_antiguos:,.0f}")
recursos_remanente_nuevos = asignacion_por_ocupacion_nuevos['Saldo_No_Asignado_2E'].sum()
print(f"Bolsa remanente Nuevos: {recursos_remanente_nuevos:,.0f}")
recursos_remanente_cerrados = Grupos_Cerrados_final.iloc[-1]["saldo_total_remanente"]
print(f"Bolsa remanente Cerrados: {recursos_remanente_cerrados:,.0f}")
bolsa_comun = recursos_remanente_antiguos + recursos_remanente_nuevos + recursos_remanente_cerrados

Bolsa remanente Antiguos: 4,390,248
Bolsa remanente Nuevos: 17,398,911
Bolsa remanente Cerrados: 3,944,691


In [79]:
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 25,733,849


In [80]:
#identificar los programas que aun no han llenado cupos:
Grupos_Cerrados_remanente = Grupos_Cerrados_final[
    Grupos_Cerrados_final['numero_cupos_ofertar'] - Grupos_Cerrados_final['cupos_asignados_2E'] > 0
].reset_index(drop=True)

Grupos_Cerrados_remanente = ordenar_sedes_programas(Grupos_Cerrados_remanente, usar_cod_cno = False)

Nuevos_remanente = Nuevos[
    Nuevos['numero_cupos_ofertar'] - Nuevos['cupos_asignados_2E'] > 0
].reset_index(drop=True)

Nuevos_remanente = ordenar_sedes_programas(Nuevos_remanente)

Antiguos_remanente = Antiguos[
    Antiguos['numero_cupos_ofertar'] - Antiguos['Total_Cupos_Asignados'] > 0
].reset_index(drop=True)

Antiguos_remanente = ordenar_ocupaciones_por_isoeft(Antiguos_remanente)

In [81]:
#llenar cupos de cerrados
Grupos_Cerrados_remanente, saldo_cerrados =  asignar_recursos_grupos_cerrados(
    Grupos_Cerrados_remanente,
    bolsa_comun
)

bolsa_comun = Grupos_Cerrados_remanente['saldo_total_remanente'].iloc[-1]

In [82]:
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 803,759


In [83]:
Nuevos_remanente, bolsa_comun = reasignar_remanente(Nuevos_remanente,bolsa_comun)
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 803,759


In [84]:
Antiguos_remanente, bolsa_comun = reasignar_remanente(Antiguos_remanente, bolsa_comun, columna_cupos = "cupos_asignados_3E")
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 803,759


---------

In [85]:
def unir_con_remanente_v2(df_original, df_remanente, llaves=['cod_CNO', 'nombre_programa', 'nombre_institucion']):
    """
    Devuelve un DataFrame combinado que prioriza las filas del original (df_original),
    especialmente el valor de 'recurso_asignado_2E', y conserva filas nuevas del remanent   
    Calcula los recursos por cno segun la formula del paso 1 de la ruta de antiguos y nuevos. 
    
    Parámetros:
        df (pd.DataFrame): DataFrame con columnas 'cod_CNO', 'Ocupacion', 'ipo_ponderado', 'numero_cupos_ofertar', 'IPO'.
        total_recursos (float): Total de recursos disponibles para distribuir.

    Retorna:
        pd.DataFrame: DataFrame con las métricas agrupadas y columna 'recursosxcno' calculada.
    """
    """
    # 1. Calcular la suma total de ipo_ponderado
    IO_total = df['ipo_ponderado'].sum()

    # 2. Agrupar por cod_CNO y Ocupacion
    IO_por_CNO = (e
    que no estaban originalmente, eliminando duplicados por llaves.

    Parámetros:
        df_original (pd.DataFrame): DataFrame original antes de la reasignación.
        df_remanente (pd.DataFrame): DataFrame después de la reasignación.
        llaves (list[str]): Columnas clave para identificar filas únicas.

    Retorna:
        pd.DataFrame: DataFrame combinado con prioridad a valores de df_original.
    """
    # Remover columna que se quiere preservar desde el original
    if 'recurso_asignado_2E' in df_remanente.columns:
        df_remanente = df_remanente.drop(columns=['recurso_asignado_2E'])

    # Concatenar remanente debajo para que df_original tenga prioridad al hacer drop_duplicates
    combinado = pd.concat([df_original, df_remanente], ignore_index=True)

    # Quitar duplicados priorizando df_original
    df_final = combinado.drop_duplicates(subset=llaves, keep='first')

    return df_final



In [86]:
def unir_con_remanente_v3(df_original, df_remanente, llaves=['cod_CNO', 'nombre_programa', 'nombre_institucion']):
    """
    Devuelve un DataFrame combinado que prioriza las filas actualizadas del remanente
    y conserva las filas no modificadas del original, sumando 'recurso_asignado_2E' y
    'cupos_asignados_2E' para filas duplicadas por llaves.

    Parámetros:
        df_original (pd.DataFrame): DataFrame original antes de la reasignación.
        df_remanente (pd.DataFrame): DataFrame después de la reasignación.
        llaves (list[str]): Columnas clave para identificar filas únicas.

    Retorna:
        pd.DataFrame: DataFrame combinado con recursos y cupos sumados en duplicados.
    """
    combinado = pd.concat([df_original, df_remanente], ignore_index=True)

    columnas_sumar = []
    if 'recurso_asignado_2E' in combinado.columns:
        columnas_sumar.append('recurso_asignado_2E')
    if 'cupos_asignados_2E' in combinado.columns:
        columnas_sumar.append('cupos_asignados_2E')

    otras_columnas = [
        col for col in combinado.columns
        if col not in llaves + columnas_sumar
    ]

    if columnas_sumar:
        agg_dict = {col: 'sum' for col in columnas_sumar}
        agg_dict.update({col: 'first' for col in otras_columnas})

        df_final = (
            combinado
            .groupby(llaves, as_index=False)
            .agg(agg_dict)
        )
    else:
        df_final = combinado.drop_duplicates(subset=llaves, keep='first')

    return df_final


In [87]:
llaves = ['cod_CNO', 'nombre_institucion','nombre_programa']

Nuevos_consolidado = unir_con_remanente_v2(Nuevos, Nuevos_remanente, llaves)
Grupos_Cerrados_consolidado = unir_con_remanente_v3(Grupos_Cerrados_final, Grupos_Cerrados_remanente, llaves = llaves)
Antiguos_consolidado = unir_con_remanente_v2(Antiguos, Antiguos_remanente, llaves)


In [88]:
#Antiguos
asignado_antiguos = Antiguos_consolidado['Total_Recurso_Asignado'].sum() + Antiguos_consolidado["recurso_asignado_remanente"].sum()
Antiguos_consolidado['total_dispersado'] = Antiguos_consolidado['Total_Recurso_Asignado'].fillna(0) + Antiguos_consolidado["recurso_asignado_remanente"].fillna(0)
Antiguos_consolidado['total_cupos_habilitados'] = Antiguos_consolidado['Total_Cupos_Asignados'].fillna(0) + Antiguos_consolidado["cupos_asignados_remanente"].fillna(0)
print(f"Asignado Antiguos: {asignado_antiguos:,.0f}")

#CERRADOS
asignado_cerrados = Grupos_Cerrados_consolidado['recurso_asignado_2E'].sum()
Grupos_Cerrados_consolidado['total_dispersado'] = Grupos_Cerrados_consolidado['recurso_asignado_2E'].fillna(0)
Grupos_Cerrados_consolidado['total_cupos_habilitados'] = Grupos_Cerrados_consolidado['cupos_asignados_2E'].fillna(0)
print(f"Asignado cerrados: {asignado_cerrados:,.0f}")

#NUEVOS
asignado_nuevos = Nuevos_consolidado["recurso_asignado_remanente"].sum()  + Nuevos_consolidado['recurso_asignado_2E'].sum()
Nuevos_consolidado['total_dispersado'] = Nuevos_consolidado["recurso_asignado_remanente"].fillna(0) + Nuevos_consolidado['recurso_asignado_2E'].fillna(0)
Nuevos_consolidado['total_cupos_habilitados'] = Nuevos_consolidado["cupos_asignados_remanente"].fillna(0) + Nuevos_consolidado['cupos_asignados_2E'].fillna(0)
print(f"Asignado nuevos: {asignado_nuevos:,.0f}")

total = asignado_antiguos +  asignado_cerrados  + asignado_nuevos
print(f"Asignado total: {total:,.0f}")

Asignado Antiguos: 1,315,609,752
Asignado cerrados: 1,010,985,400
Asignado nuevos: 972,601,089
Asignado total: 3,299,196,241


------

In [89]:
Grupos_Cerrados_consolidado.to_excel("Export/cerrados_consolidado.xlsx", index=False)
Antiguos_consolidado.to_excel("Export/antiguos_consolidado.xlsx", index=False)
Nuevos_consolidado.to_excel("Export/nuevos_consolidado.xlsx", index=False)

In [95]:
# Obtener las columnas comunes entre todos los DataFrames
#columnas_exportar = columnas_exportar = list(Programas_EFT.columns) + ['total_dispersado', 'total_cupos_habilitados']

columnas_exportar = [
    'Ruta habilitada',
    'nombre_institucion',
    'cod_CNO',
    'Ocupacion',
    'nombre_programa',
    "CODIGO_PROGRAMA",
    'IPO',
    'ISOEFT_4d',
    'Puntaje (nuevos y cerrados)',
    'Meta de vinculación',
    columna_valor_programa,
    "numero_cupos_ofertar",
    'duracion_horas_programa',
    'total_dispersado',
    'total_cupos_habilitados',
    "Número máximo de cupos por grupos",
]

# Filtrar solo esas columnas en cada DataFrame
df_grupos = Grupos_Cerrados_consolidado[columnas_exportar]
df_antiguos = Antiguos_consolidado[columnas_exportar]
df_nuevos = Nuevos_consolidado[columnas_exportar]

# Concatenar
df_consolidado = pd.concat([df_grupos, df_antiguos, df_nuevos], ignore_index=True)

# Guardar en distintas hojas del mismo Excel
with pd.ExcelWriter("Export/Dispersiones_EFT_ciclolargo_primera_convocatoria.xlsx", engine='openpyxl') as writer:
    df_consolidado.to_excel(writer, sheet_name='Todos', index=False)
    df_grupos.to_excel(writer, sheet_name='Grupos Cerrados', index=False)
    df_antiguos.to_excel(writer, sheet_name='Antiguos', index=False)
    df_nuevos.to_excel(writer, sheet_name='Nuevos', index=False)

In [96]:
total_dispersado = df_consolidado['total_dispersado'].sum()
print(f"Total dispersado: {total_dispersado:,.0f}")

Total dispersado: 3,299,196,241


In [97]:
total_cupos = df_consolidado['total_cupos_habilitados'].sum()
print(f"total_cupos: {total_cupos:,.0f}")

total_cupos: 642


-----

El siguiente dataframe es un resumen de 1 de las 3 alternativas de asignación:

Este corresponde a la alternativa de "Convexa"

In [98]:
resumen = df_consolidado.groupby(
    by=["Ruta habilitada", "nombre_institucion", "nombre_programa","Ocupacion"]
).agg(
    ISOEFT=("ISOEFT_4d", "first"),
    IPO=("IPO", "first"),
    cupos_habilitados=("total_cupos_habilitados", "sum"),
    monto_asignado = ("total_dispersado","sum")
).reset_index()

resumen["ISOEFT"] = resumen["ISOEFT"].fillna("NA")
# Filtrar las filas donde cupos_habilitados > 0
resumen = resumen[resumen["cupos_habilitados"] > 0]

resumen = resumen.sort_values(
    by=["Ruta habilitada", "cupos_habilitados"], ascending=[True, False]
).reset_index(drop=True)

# Convertir a minúsculas y luego capitalizar la primera letra
resumen["nombre_institucion"] = resumen["nombre_institucion"].str.lower().str.capitalize()
resumen["nombre_programa"] = resumen["nombre_programa"].str.lower().str.capitalize()


resumen.to_excel("Export/resumen_resultados.xlsx")

In [99]:
resumen[['Ruta habilitada','Ocupacion','IPO','cupos_habilitados','monto_asignado']].to_excel("Export/resumen_via_original.xlsx")